In [1]:
%%HTML
<style>
   div#notebook-container    { width: 95%; }
   div#menubar-container     { width: 65%; }
   div#maintoolbar-container { width: 99%; }
</style>

In [2]:
# import sys
# !{sys.executable} -m pip install torchsummary

In [1]:
import sys
sys.path.append("..") # Adds higher directory to python modules path.
from utilities import aggregate_feature_calculators
from utilities import aggregate_feature_calculators_setting as aggcal
from utilities.parallel import Parallel

In [2]:
import pandas as pd
import numpy as np
import os
import os.path as osp
import datetime

import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm_notebook

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold, TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split

from torch_geometric.data import Data, DataLoader
import torch
import torch.nn.functional as F
from torch_geometric.datasets import Planetoid
from torch_geometric.datasets import QM9
from torch_geometric.datasets import TUDataset
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv, ChebConv  # noqa
from torch.nn import Sequential, Linear, ReLU, GRU
import torch_geometric.transforms as T
from torch_geometric.datasets import QM9
from torch_geometric.nn import NNConv, Set2Set
from torch_geometric.data import DataLoader
from torch_geometric.utils import remove_self_loops
from torch_geometric.data import Data

In [ ]:
file_folder = 'data/input/champs-scalar-coupling' if 'champs-scalar-coupling' in os.listdir('../data/input/') else '../data/input'
os.listdir(file_folder)

In [219]:
train = pd.read_csv(f'{file_folder}/train.csv')
test = pd.read_csv(f'{file_folder}/test.csv')
sub = pd.read_csv(f'{file_folder}/sample_submission.csv')
train['type0'] = train['type'].apply(lambda x: int(x[0]))
train['type1'] = train['type'].apply(lambda x: x[1:])

le = LabelEncoder()
le.fit(train.type1.tolist())
int_bond_type = le.transform(train.type1.tolist()) 
train['int_type1']= int_bond_type

test['type0'] = test['type'].apply(lambda x: int(x[0]))
test['type1'] = test['type'].apply(lambda x: x[1:])
int_bond_type = le.transform(test.type1.tolist()) 
test['int_type1']= int_bond_type

In [158]:
magnetic_shielding_tensors = pd.read_csv(f'{file_folder}/magnetic_shielding_tensors.csv')
dipole_moments = pd.read_csv(f'{file_folder}/dipole_moments.csv')
mulliken_charges = pd.read_csv(f'{file_folder}/mulliken_charges.csv')
potential_energy = pd.read_csv(f'{file_folder}/potential_energy.csv')
scalar_coupling_contributions = pd.read_csv(f'{file_folder}/scalar_coupling_contributions.csv')

In [166]:
structures = pd.read_csv(f'{file_folder}/structures.csv')
le = LabelEncoder()
le.fit(structures.atom.tolist())
int_atom = le.transform(structures.atom.tolist()) 
structures['int_atom'] = int_atom

In [ ]:
molecule_names = train.molecule_name.unique().tolist()

In [204]:
magnetic_shielding_tensors.head()#MAE 1000+

,molecule_name,atom_index,XX,YX,ZX,XY,YY,ZY,XZ,YZ,ZZ
0,dsgdb9nsd_000001,0,195.3150,0.0000,-0.0001,0.0000,195.3170,0.0007,-0.0001,0.0007,195.3170
1,dsgdb9nsd_000001,1,31.3410,-1.2317,4.0544,-1.2317,28.9546,-1.7173,4.0546,-1.7173,34.0861
2,dsgdb9nsd_000001,2,31.5814,1.2173,-4.1474,1.2173,28.9036,-1.6036,-4.1476,-1.6036,33.8967
3,dsgdb9nsd_000001,3,31.5172,4.1086,1.2723,4.1088,33.9068,1.6950,1.2724,1.6951,28.9579
4,dsgdb9nsd_000001,4,31.4029,-4.0942,-1.1793,-4.0944,34.0776,1.6259,-1.1795,1.6260,28.9013


In [159]:
dipole_moments.head()#MAE 3.x without scaled y, 2.x with scaled y

,molecule_name,X,Y,Z
0,dsgdb9nsd_000001,0.0000,0.0,0.0000
1,dsgdb9nsd_000002,-0.0002,0.0,1.6256
2,dsgdb9nsd_000003,0.0000,0.0,-1.8511
3,dsgdb9nsd_000005,0.0000,0.0,-2.8937
4,dsgdb9nsd_000007,0.0000,0.0,0.0000


In [203]:
mulliken_charges.head()#MAE 0.4x with scaled y

,molecule_name,atom_index,mulliken_charge
0,dsgdb9nsd_000001,0,-0.535689
1,dsgdb9nsd_000001,1,0.133921
2,dsgdb9nsd_000001,2,0.133922
3,dsgdb9nsd_000001,3,0.133923
4,dsgdb9nsd_000001,4,0.133923


In [253]:
potential_energy.head()#MAE 30.x with scaled y

,molecule_name,potential_energy
0,dsgdb9nsd_000001,-40.52368
1,dsgdb9nsd_000002,-56.56025
2,dsgdb9nsd_000003,-76.42608
3,dsgdb9nsd_000005,-93.42849
4,dsgdb9nsd_000007,-79.83869


In [268]:
scalar_coupling_contributions.head()

,molecule_name,atom_index_0,atom_index_1,type,fc,sd,pso,dso
0,dsgdb9nsd_000001,1,0,1JHC,83.0224,0.254579,1.25862,0.272010
1,dsgdb9nsd_000001,1,2,2JHH,-11.0347,0.352978,2.85839,-3.433600
2,dsgdb9nsd_000001,1,3,2JHH,-11.0325,0.352944,2.85852,-3.433870
3,dsgdb9nsd_000001,1,4,2JHH,-11.0319,0.352934,2.85855,-3.433930
4,dsgdb9nsd_000001,2,0,1JHC,83.0222,0.254585,1.25861,0.272013


In [255]:
y_std = np.std(potential_energy[['potential_energy']].values, axis=0)
y_mean = np.std(potential_energy[['potential_energy']].values, axis=0)
y_std, y_mean

(array([39.82863665]), array([39.82863665]))

In [256]:
def _worker(item, df_bonds, df_structures, potential_energy):
    
    has_y = True
    idx = item[0]
    molecule_name = item[1]
    x = torch.tensor(df_structures[df_structures['molecule_name']==molecule_name].sort_values(by=['atom_index'])[['int_atom']].values, dtype=torch.float)
    pos = torch.tensor(df_structures[df_structures['molecule_name']==molecule_name].sort_values(by=['atom_index'])[['x','y','z']].values, dtype=torch.float)

    edge_index = []
    edge_attr = []

    df_molecule_i = df_bonds[df_bonds['molecule_name']==molecule_name]
    potential_energy_i = potential_energy[potential_energy['molecule_name']==molecule_name]

    if has_y:
        y = (potential_energy_i[['potential_energy']].values-y_mean)/y_std
        y = torch.tensor(y, dtype=torch.float)

    for idx, row in df_molecule_i.iterrows():
        edge_index.append([row['atom_index_0'], row['atom_index_1']])
        edge_index.append([row['atom_index_1'], row['atom_index_0']])
        edge_attr.append([row['type0'], row['int_type1']])
        edge_attr.append([row['type0'], row['int_type1']])

    edge_index = torch.tensor(edge_index, dtype=torch.long)
    edge_attr = torch.tensor(edge_attr, dtype=torch.float)
    if has_y:
        data = Data(x=x, edge_index=edge_index.t().contiguous(), pos=pos, edge_attr=edge_attr, y=y)
    else:
        data = Data(x=x, edge_index=edge_index.t().contiguous(), pos=pos, edge_attr=edge_attr)
    return {'idx':idx, 'molecule_name':molecule_name, 'data':data}

def generate_datalist(df_bonds, df_structures, molecule_names=['dsgdb9nsd_000007', 'dsgdb9nsd_000008']):

    N_ = len(molecule_names)
#     segment = N_//100
    data_list = []
    for item in tqdm_notebook(enumerate(molecule_names), total=N_):
        d_ = _worker(item, **{'df_bonds':df_bonds, 'df_structures':df_structures, 'potential_energy':potential_energy})
        data_list.append(d_)
        
    df_ = pd.DataFrame(data_list)
    df_ = df_.sort_values(by=['idx'])
    df_ = df_.reset_index(drop=True)
#     df_.to_pickle(path)
    return df_
    

In [257]:
df_data_train = generate_datalist(train, structures, molecule_names=molecule_names[:1000])
df_data_train = df_data_train.drop(columns=['idx'])

In [259]:
df_data_train.data.tolist()[0]

Data(edge_attr=[20, 2], edge_index=[2, 20], pos=[5, 3], x=[5, 1], y=[1, 1])

In [260]:
train_molecule_names, valid_molecule_names =  train_test_split(molecule_names, train_size=.8, shuffle=True)

/home/ubuntu/anaconda3/envs/tensorflow_gpu_p36/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2179: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [261]:
def generate_dataLoader(df, molecule_names, shuffle=True, batch_size=32, **kwargs):
    data_list = df[df['molecule_name'].isin(molecule_names)]['data'].tolist()
    return DataLoader(data_list, batch_size=batch_size, shuffle=shuffle, **kwargs)

In [262]:
# test_loader = generate_dataLoader(df_data_train, test_molecule_names)
val_loader = generate_dataLoader(df_data_train, valid_molecule_names)
train_loader = generate_dataLoader(df_data_train, train_molecule_names)

In [266]:
dim = 64


class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.lin0 = torch.nn.Linear(1, dim)

        nn = Sequential(Linear(2, 128), ReLU(), Linear(128, dim * dim))
        self.conv = NNConv(dim, dim, nn, aggr='mean', root_weight=False)
        self.gru = GRU(dim, dim)

        self.set2set = Set2Set(dim, processing_steps=3)
        self.lin3 = torch.nn.Linear(dim, 2 * dim)
        self.lin1 = torch.nn.Linear(2 * dim, dim)
        self.lin2 = torch.nn.Linear(dim, 1)

    def forward(self, data):
        out = F.relu(self.lin0(data.x))
        h = out.unsqueeze(0)
        for i in range(3):
            m = F.relu(self.conv(out, data.edge_index, data.edge_attr))
            out, h = self.gru(m.unsqueeze(0), h)
            out = out.squeeze(0)
            
        out = self.set2set(out, data.batch)
#         out = F.relu(self.lin3(out))
        out = F.relu(self.lin1(out))
        out = self.lin2(out)
#         print(out.shape)
#         out = out.view(-1)
        return out


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = Net().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.7, patience=5, min_lr=0.00001)


def _train(epoch, loader):
    model.train()
    loss_all = 0

    for data in loader:
        data = data.to(device)
        optimizer.zero_grad()
        y_pred = model(data)
        loss = F.mse_loss(y_pred, data.y)
        loss.backward()
        loss_all += loss.item() * data.num_graphs
        optimizer.step()
    return loss_all / len(loader.dataset)


def _test(loader):
    model.eval()
    error = 0

    for data in loader:
        data = data.to(device)
#         error += (model(data) - data.y).abs().sum().item()
        std = torch.tensor(y_std.reshape(1,1), dtype=torch.float)
        std = std.to(device)
        error += (model(data).mm(std) - data.y.mm(std)).abs().sum().item()  # MAE
    return error / len(loader.dataset)

In [267]:
best_val_error = None
for epoch in range(1, 101):
    lr = scheduler.optimizer.param_groups[0]['lr']
    loss = _train(epoch, train_loader)
    val_error = _test(val_loader)
    scheduler.step(val_error)
    print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}'.format(epoch, lr, loss, val_error))

#     if best_val_error is None or val_error <= best_val_error:
#         test_error = test(test_loader)
#         best_val_error = val_error

#     print('Epoch: {:03d}, LR: {:7f}, Loss: {:.7f}, Validation MAE: {:.7f}, Test MAE: {:.7f}'.format(epoch, lr, loss, val_error, test_error))

Epoch: 001, LR: 0.001000, Loss: 37.3571763, Validation MAE: 54.2919280
Epoch: 002, LR: 0.001000, Loss: 2.3483029, Validation MAE: 43.4473093
Epoch: 003, LR: 0.001000, Loss: 1.6934202, Validation MAE: 40.6297085
Epoch: 004, LR: 0.001000, Loss: 1.5899585, Validation MAE: 39.8087050
Epoch: 005, LR: 0.001000, Loss: 1.5709239, Validation MAE: 39.7497070
Epoch: 006, LR: 0.001000, Loss: 1.5670594, Validation MAE: 40.5006430
Epoch: 007, LR: 0.001000, Loss: 1.5601451, Validation MAE: 39.7622238
Epoch: 008, LR: 0.001000, Loss: 1.5496617, Validation MAE: 39.4713419
Epoch: 009, LR: 0.001000, Loss: 1.5648724, Validation MAE: 40.3552394
Epoch: 010, LR: 0.001000, Loss: 1.5281130, Validation MAE: 39.5524380
Epoch: 011, LR: 0.001000, Loss: 1.5247856, Validation MAE: 39.4386594
Epoch: 012, LR: 0.001000, Loss: 1.5604966, Validation MAE: 40.1909668
Epoch: 013, LR: 0.001000, Loss: 1.5585748, Validation MAE: 41.1110641
Epoch: 014, LR: 0.001000, Loss: 1.5374452, Validation MAE: 39.5729708
Epoch: 015, LR: 0.0

In [ ]:
Epoch: 100, LR: 0.000343, Loss: 0.0081743, Validation MAE: 0.3129633, Test MAE: 0.3087115　.6
Epoch: 100, LR: 0.000010, Loss: 0.0050267, Validation MAE: 0.2659131, Test MAE: 0.2802887   .8